# Feature Extraction
    이미지 인식을 위해 뽑아내야 하는 low level 의 정보들을 뽑아내는 것

# CNN operation
    target 이미지와 동일한 패턴을 찾아내는 것이 아니라 target 이미지가 지니는 부분적인 패턴들을 정의하고 그 패턴들을 지니고 있는지를 확인함.
	그 패턴을 convolutional filter이라고 하고 입력 데이터를 필터가 순회하며 픽셀끼리의 곱을 합산하여(내적) 필터의 픽셀의 수로 나눈다.
	필터가 해당이미지를 모두 순회하면 내적의 평균값들이 필터가 순회한 만큼 나온다. 그것을 mapping 하여 새로운 map을 생성한 것이 activation map이라고 함
	.77 -> 77% 매칭. 1 -> 100% 매칭
	activation map의 값 하나하나는 선형결합으로 생겨난 것. 예) activation map의 첫번째 값은 convolutional filter와 이미지 맨왼쪽맨위의 9개의(예를 들면) 픽셀과의 내적의 합의 평균의 결과이다.
	따라서 이것을 neural net으로 생각해보면 입력노드는 이미지의 9개의 픽셀이고 가중치는 convolutional filter이고 출력노드는 activation map의 값 하나이다.
	-> convolution layer
	convolutional filter는 3*3이지만 RGB이므로 3개가 쌓여있다. 쌓여있는 filter을 activation map에 순회를 하면 단순히 처음의 출력값보다 주변 픽셀들의 값을 더 고려한 high level 의 정보를 담게 된다.
	hidden layer가 쌓일수록 주변 픽셀들의 정보를 고려하게 되므로 점점 이미지를 잘 맞출 수 있게 되는 것이다.
	conv-relu-pooling 의 과정이 일반적인 CNN의 운영과정이다. 합성곱 진행후 RELU 활성화함수 통과 Pooling 진행

# Max Pooling
    activation map을 특정 window크기로 순회하며 가장 큰 값만을 뽑아냄. 가장 우세한 값을 뽑아냄으로써 target이미지와 완벽히 일치하지 않아도 
	해당 패턴을 지니는 위치가 비슷하고 그 정도의 세기로 나타냈음을 알려주게된다. 전체 이미지 사이즈도 줄게 됨. 
	activation map이 max pooling을 통해 출력을 내는 층을 pooling layer 라고 함

# Pre-trained Model
    이미 모델구조가 fix되어있고 최적화된 파라미터가 있는 상태에서 초반 레이어는 edge정도를 생성하기 때문에 대부분 task에서 재사용될 수 있음.
	추후에 본인의 task에 맞게끔 pre-trained model에 hidden layer을 추가하고 그 부분의 파라미터만 학습시켜 task를 수행하는 것을 transfer learning(전이 학습)이라고 한다.

# VGG-network
    100*100*64(depth)의 input image를 100*100*64의 output 으로 하려면 filter의 파라미터의 수는 가로세로를 7*7로 한다면 7*7*64 여기까지가 input을 순회하는 한개의 필터고, 필터가 64개가 필요하므로 총 파라미터는 7*7*64*64가 된다.
	3*3*depth의 filter을 여러번 거치면 input image에 대해서는 7*7*depth 정도의 receptive field를 커버할 수 있다(범위가 계속 누적되므로). 
	처음부터 7*7*depth filter을 쓰는 것 보다 3*3*depth filter을 쓰고 layer을 늘리는 것이 파라미터 수도 적고 (3*3*64 여기까지가 한개의 필터 3*3*64*64 (64개의 필터가 필요하므로) 여기에 layer가 3이라면 3*3*64*64*3이 된다.) 
	중간에 non linearity를 활용할 수 있으므로 VGG-net이 낫다.
	위의 파라미터 연산이 중요함. input 이미지를 순회하는 필터의 가로세로를 n,m이라고 하고 input 이미지의 depth를 C라고 하고 필터의 개수를 C'라고 하면
	필요한 파라미터는 n*m*C*C'이 된다. conv후의 output인 activation map의 크기는 가로세로 설정에 따라 다른데, 가로*세로*필터의 개수이다.

# Google-net
    1*1 convolution: demension을 줄여주는 pre process로 많이 쓰임(파라미터 수 감소 가능)
		주변 픽셀들을 아우르지 않고 1*1이므로 하나의 픽셀만 뽑아내어 필터의 개수로 
		output activation map의 depth를 조절 가능
		bottle neck 구조로 사용가능 : depth가 64인 map을 1*1 conv로 depth를 10으로 줄이고 
		3*3conv(이 때 필요한 feature을 추출)로 다시 depth를 10으로 맞추고 depth를 복원하기 위해 
		다시 1*1conv로 depth를 64로 늘릴 수 있음.
	pooling 과정에서도 가로세로를 줄이지 않고 stride와 padding을 조절하여 가로세로 유지 가능
	Inception module: 가로세로가 동일한 여러 activation map을 다시 concatenate하여 depth가 깊은 output 생성

	Multi-task Learning: 학습을 진행하다가 두갈래로 output을 나눔
		-> 두갈래로 나누기 전까지는 일반적인 feature을 뽑고 갈래로 들어갈수록 specific한 특성을 뽑아냄
		미리 한갈래를 빼놔서 fully connected로 좀 이르게 classification을 진행
			-> 이렇게 진행하면 back prop을 미리 빼놓은 갈래길에서 진행하게 되고 가중치 학습을 먼저 함.
			갈래를 빼지 않고 전체 network을 back prop 통째로 하면 gradient vanishing이나 
			기울기가 변질될 가능성이 있는데
			갈래를 하나씩 빼놓고 gradient 를 update하면 미리 edge를 만들어놓고 
			나중에 재조합할 때 도움을 줄 수 있다는 의도
			-> 요즘은 batch norm이나 adam 등으로 갈래를 안빼놔도 통째로 back prop해도 괜찮게 나옴
		두갈래를 한번에 back prop 해서 두마리 토끼를 잡을 수 있는 multi task learning 도 가능 
		-> 여러 패턴을 한번에 고려하면 overfitting등도 예방할 수도 있음
		다른 장점으로는 갈래별로 데이터 개수의 차이가 있는 상황에서는 
		갈래길전에 공통된 경로에서 더 많이 학습 가능함.
		연관이 없어보여도 전반적인 공통적인 패턴을 뽑아낼 수 있음. 
		현재 구글 번역기의 근간이 되는 idea임. 
		한국어에서 우간다어로 번역하는 그런 데이터의 수가 적은 경우에 대해 
		multi task learning 기반의 모듈을 사용하므로 갈래가 갈라지기 전에 
		언어들 전체에 일반적으로 흐르는 패턴을 우선 학습하여 데이터의 부족을 매꿀 수 있음.

	Fully-connected Layer가 쓰이지 않고 convolution layer로 대체 -> activation map을 아예 덮는 convolution filter로 map을 찍어내면 하나의 값이 나옴. FC에서 예를 들어 4096개로 펼치는 작업을 convolution filter을 4096개 사용하여 하나의 픽셀의 depth를 4096으로 만들어 FC를 대체할 수 있음

# Res-Net
    층이 깊어질수록 학습데이터에 대해서 성능이 올라가야하지만, 층이 깊어질수록 학습데이터에 대한 성능이 오히려 떨어지는 결과가 나타남 -> back prop에서 gradient vanishing의 문제가 있음, 활성화함수가 쌓일수록 항등함수의 역할을 할 수 없고 그렇기 때문에 layer가 적었을 때와는 결과가 달라질 수 있음

    이 문제를 해결하기 위해 res-net이 나옴
    출력인 h(x)를 기존의 conv-relu의 f(x)에 input값 x를 더하여 f(x)+x = h(x) 의 꼴로 만든 모듈을 사용.
    이렇게 하면 f(x)를 h(x) - x 즉, 잔차에 맞추기에 집중할 수 있음. 원래의 값보다 작은 값을 만들기 때문에 안정적임(논리적 비약이 있음). 또한 위의 항등함수의 역할을 하기 위해 f를 0으로 처리하면 h(X) = x가 되므로 항등함수로써의 역할도 수행가능.
    따라서 층이 여러개일때가 층이 적었을 때를 포괄하는 개념이 가능해짐.

    x를 h로 흘려보내는 것을 skip connection이라고 함.

    bottle neck을 활용하여 연산을 도움


# Dense-net
    input 데이터를 출력데이터에 concatnate해줌. res-net과 다르게 더하는게 dense-net은 더할때 뒤에 척 붙이듯이 depth방향으로 길게 늘어뜨려 붙이는 방식임. 뒤에 출력에 더 다양한 정보를 담을 수 있음. 같은 level의 조합만이 아닌, high level의 정보와 low level의 정보를 붙여서 다양한 패턴의 조합을 가능케한다. 

    dense block에서 bottle neck 구조로 연산에 도움을 줌(dense net에서는 필수임)

Res-net과 Dense-net 모두 더하기를 기본으로 하므로 back prop에서 sigmoid나 tanh, relu등의 기울기를 소실할 가능성을 이겨낼 수 있음.(합성함수 미분에 의해 더하기가 있으면 기울기가 그대로 전달되는 부분이 생김)